## Bigeard Jean-Jules

In [5]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [6]:
!pip install --upgrade git+git://github.com/frehseg/nvcc4jupyter.git

  Cloning git://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-gp5il8ll
  Running command git clone -q git://github.com/frehseg/nvcc4jupyter.git /tmp/pip-req-build-gp5il8ll
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.1-cp36-none-any.whl size=2095 sha256=2800c7a0d1bd2316fcecb7d1c087c45419870f39bbbfec485065243db6e4db40
  Stored in directory: /tmp/pip-ephem-wheel-cache-_x9un42q/wheels/a4/a5/24/17a2b61f9a725a10155cc6fca753aae28436921df21fa16114
Successfully built NVCCPlugin
  Found existing installation: NVCCPlugin 0.0.1
    Uninstalling NVCCPlugin-0.0.1:
      Successfully uninstalled NVCCPlugin-0.0.1


In [8]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [79]:
%%cu 
/** CUBLAS example taken from http://courses.cms.caltech.edu/cs179/ */

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#include <iostream>
#include <iomanip>
#include <math.h>
#include <fstream>

/* Includes, cuda */
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <helper_cuda.h>

using namespace std;


/* transform matrix index to vector offset
   Since CUDA uses column major, 
   ld = number of rows 
   Example of use: a[IDX2C(0, 1, 50)] */
#define IDX2C(i,j,ld) (((j)*(ld))+(i))

/* Constants for housing data set */
#define data_columns  (9)
#define above_threshold (265000.0)

/////////////////////////////////////////////////////////
// Number of rows in arrays to print for debugging
/////////////////////////////////////////////////////////
#define print_rows (10)

/////////////////////////////////////////////////////////
// Auxiliary function
/////////////////////////////////////////////////////////
// generate random numbers in interval [min,max]
float float_rand( float min, float max )
{
    float scale = rand() / (float) RAND_MAX; /* [0, 1.0] */
    return min + scale * ( max - min );      /* [min, max] */
}


/////////////////////////////////////////////////////////
// Functions for reading the dataset from a file
/////////////////////////////////////////////////////////

/* Read a csv file with a given number of rows and columns */
void read_csv(const char* filename, float* data_array,size_t nbrow,size_t nbcol) {
  string row_as_string;
  string value;
  double ioTemp;
  ifstream infile;
  infile.open(filename, ifstream::in);
  size_t row_count = 0;
	if (infile.is_open())
  {
      // read the headers (and discard)
			getline(infile, row_as_string, '\n');
      cout << "headers: " << row_as_string << "!" << std::endl;
      for(int i = 0; i < nbrow; i++){
  			getline(infile, row_as_string, '\n');
        // cout << "read line " << row_as_string << "!" << std::endl;
				istringstream line_stream(row_as_string);
			  for(int j = 0; j < nbcol; j++){
          getline(line_stream, value, ',');
					ioTemp = strtod(value.c_str(), NULL); 
          // cout << "("<<i<<","<<j<<") = "<< ioTemp << std::endl;

					data_array[IDX2C(i,j,nbrow)] = ioTemp;

				}
        ++row_count;
			}
		infile.close();
    cout << "Read " << row_count << " rows." << std::endl;
	}
	else cout << "Cannot open file." << endl;
}

/* Allocate memory and fill with given number of rows from housing file */
void read_housing_csv(float** data_array, size_t nbrows) {
    *data_array = (float *)malloc(nbrows * data_columns * sizeof(float));
    
    read_csv("sample_data/california_housing_train.csv",*data_array,nbrows,data_columns);

    cout << "Data (first "<<print_rows<<" rows):" << std::endl;
    // Show some entries for double checking
    for(int i = 0; i < nbrows && i < print_rows; i++){
			for(int j = 0; j < data_columns; j++){
				cout << (*data_array)[IDX2C(i,j,nbrows)] << "\t";
			}
      cout << "\n";
		}
}

/////////////////////////////////////////////////////////
// Functions for preprocessing the data set
/////////////////////////////////////////////////////////

/* Split data into inputs and labels. Allocated memory for inputs and labels.
   Since cuBLAS is column major, each input is in a column.
   We also add 1.0 as first element to each input vector.
*/
void get_inputs_and_labels(float* data_array, float** input_array, float** label_array, size_t nbrows, size_t nbcols, size_t* nb_inputs, size_t* nb_labels ) {
    // The inputs are the first nbrows-1 columns.
    // The labels are the last column (index nbrows-1), booleanized
    // by the condition >= above_threshold
    size_t nbcols_input = nbcols-1+1; // remove one column, add one for 1.0
    size_t nbcols_labels = 2;
    *nb_inputs = nbcols_input;
    *nb_labels = nbcols_labels;
    *input_array = (float *)malloc(nbrows * nbcols_input * sizeof(float));    
    *label_array = (float *)malloc(nbrows * nbcols_labels * sizeof(float));    
    //cout << &input_array << " and "<< &label_array << " data " << data_array << std::endl;
    cout << "Allocated memory: " << nbrows << " rows, "<< nbcols_input << " columns." << std::endl;

    // Copy the data to X
    for(int i = 0; i < nbrows; i++){
      // Set the first element of each x to 1  
      (*input_array)[IDX2C(0,i,nbcols_input)] = 1.0;
      // Copy the rest of x
			for(int j = 1; j < nbcols_input; j++){
				(*input_array)[IDX2C(j,i,nbcols_input)] = data_array[IDX2C(i,j-1,nbrows)];
			}
      float median_house_value = data_array[IDX2C(i,nbcols-1,nbrows)];
      (*label_array)[IDX2C(i,0,nbrows)] = 0.0;
      (*label_array)[IDX2C(i,1,nbrows)] = 0.0;
      if (median_house_value >= above_threshold) {
        (*label_array)[IDX2C(i,0,nbrows)] = 1.0;
      } else {
        (*label_array)[IDX2C(i,1,nbrows)] = 1.0;        
      }
		}    
    
    // Show some entries for double checking
    cout << "Inputs (first "<<print_rows<<"):" << std::endl;
	  for(int j = 0; j < nbcols_input; j++){
      for(int i = 0; i < nbrows && i < print_rows; i++){
				cout << (*input_array)[IDX2C(j,i,nbcols_input)] << "\t";
			}
      cout << "\n";
		}
    cout << "Labels (first "<<print_rows<<"):" << std::endl;
    for(int i = 0; i < nbrows && i < print_rows; i++){
			for(int j = 0; j < nbcols_labels; j++){
				cout << (*label_array)[IDX2C(i,j,nbrows)] << "\t";
			}
      cout << "\n";
		}
}


/////////////////////////////////////////////////////////
// Main program
/////////////////////////////////////////////////////////
int main(int argc, char **argv)
{
    
    /////////////////////////////////////////////////////////
    // Parameters for the data set
    /////////////////////////////////////////////////////////
    size_t N_train = 2000; // points for training (Google: 12000)
    size_t N_test = 1000; // points for validation (Google: 5000)
    size_t N = N_train+N_test;
    /////////////////////////////////////////////////////////
    // Reading the data set
    /////////////////////////////////////////////////////////
    float *alldata = 0;
    read_housing_csv(&alldata,N);
    float* X = 0; 
    float* Y = 0;
    size_t D; // number of inputs
    size_t M; // number of labels
    get_inputs_and_labels(alldata,&X,&Y,N,data_columns,&D,&M);
    /////////////////////////////////////////////////////////
    // Inputs and labels are now available in X and Y.
    // Each input is a column in X; each feature is a row;
    //    by convention, the first row has the value 1.0;
    //    X is of dimension D x N
    // each label is a row in Y; Y is of dimension N x M
    /////////////////////////////////////////////////////////

    
    /////////////////////////////////////////////////////////
    // Parameters for Stochastic Gradient Descent
    /////////////////////////////////////////////////////////
    int nb_iter = 10;
    int periods = nb_iter; // reporting periods
    float step_size = 0.000001;
    
    
    /////////////////////////////////////////////////////////
    // Memory Allocation and Initialization
    /////////////////////////////////////////////////////////
    cublasStatus_t status;
    float *h_W;
    float *h_Z;
    float *h_q;
    float *d_X = 0;
    float *d_W = 0;
    float *d_Z = 0;
    float *d_q = 0;
    float alpha = 1.0f;
    float beta = 0.0f;
    size_t nX = D * N; // nb of elements in X
    size_t nW = D * M; // nb of elements in W
    size_t nZ = M * N; // nb of elements in Z=W^TX
    int i,j,k;
    float J;
    cublasHandle_t handle;

    status = cublasCreate(&handle);

    /* Allocate host memory for the matrices */
    h_Z = (float *)malloc(nZ * sizeof(h_Z[0]));
    h_q = (float *)malloc(M * sizeof(h_q[0]));

    /////////////////////////////////////////////////////////
    // Initializing Weight Matrix 
    // its dimension is D x M
    /////////////////////////////////////////////////////////
    /* Find scale for each input */
    float scale[D];
    for (int i = 0; i < D; ++i) {
        scale[i] = 0.0;
        for (int j = 0; j < N_train; ++j) {
            scale[i] = max(scale[i],abs(X[IDX2C(i,j,D)]));
        }
    }

    /* Initialize Weight Matrix,
       scaling each row according to the magnitude of its
       corresponding feature.
       Its dimension is D * M */
    h_W = (float *)malloc(nW * sizeof(h_W[0]));
    for (int i = 0; i < D; ++i) {
        for (int j = 0; j < M; ++j) {
            h_W[IDX2C(i,j,D)] = 1.0/sqrt(D) * float_rand( -1, 1 ) / scale[i];
        }
    }

    /* Allocate device memory for the matrices */
    cudaMalloc((void **)&d_Z, nZ * sizeof(d_Z[0]));
    cudaMalloc((void **)&d_X, nX * sizeof(d_X[0]));
    cudaMalloc((void **)&d_W, nW * sizeof(d_W[0]));
    cudaMalloc((void **)&d_q, M * sizeof(d_q[0]));

    /* Initialize the device matrices with the host matrices */
    status = cublasSetVector(nX, sizeof(X[0]), X, 1, d_X, 1);
    status = cublasSetVector(nW, sizeof(h_W[0]), h_W, 1, d_W, 1);

    /////////////////////////////////////////////////////////
    // Batch Gradient Descent
    /////////////////////////////////////////////////////////
    for (i = 0; i < nb_iter; ++i ) {
        ////////////////////////////////
        // compute Z = W^T X
        // --> each column z of Z corresponds to one column x of X
        ////////////////////////////////
        // Reminder: 
        // W^T has dimensions M by D
        // X has dimensions D by N
        // Z has dimensions M by N
        
        /* ... to be completed ... */

        status = cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_N, 
                             M, N, D, &alpha, 
                             d_W, D, d_X, D, &beta, d_Z, M);

        ////////////////////////////////
        // Compute softmax activation p(z)
        // Version 1: on the host
        ////////////////////////////////
        // Copy Z back to host
        status = cublasGetVector(nZ, sizeof(h_Z[0]), d_Z, 1, h_Z, 1);
          
        ////////////////////////////////
        // For each column z of Z, compute p;
        // then update W
        ////////////////////////////////
        #include <time.h>
        clock_t start,end;
        start = clock();
        J = 0.0;
        for (j=0; j < N_train; ++j) {
          ////////////////////////////////
          // For the k-th element of z,
          // compute p_k = exp(z_k)/sum_t exp(z_t)
          //         q_k = p_k - y_k(j)
          ////////////////////////////////
        
          /* ... compute q_k and store in array h_q[k] ...
            ... to be completed ... */

          float sumz = 0.0;       
             
          for (k=0; k < M; ++k) {
             sumz += exp(h_Z[IDX2C(k,j,M)]);
          }
          for (k=0; k < M; ++k) {
             float p_k = exp(h_Z[IDX2C(k,j,M)])/sumz;
             float q_k = p_k - Y[IDX2C(j,k,N)];
             h_q[k] = q_k;
             J -= Y[IDX2C(j,k,N)]*log(p_k);}
            
          // Copy q onto device
          status = cublasSetVector(M, sizeof(h_q[0]), h_q, 1, d_q, 1);
            
          ////////////////////////////////
          // Compute the LogLoss score
          // J = - sum_j sum_k y_k * log(p_k) / N_train
          ////////////////////////////////
            
          ////////////////////////////////
          // compute  d = x(p-y)^T;
          // update   W = W - a/N_train * d
          ////////////////////////////////
  
          /* ... to be completed ... */

          alpha = - step_size / N_train ;
          beta = 1;

          status = cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_T, 
                             D, M, 1, &alpha, 
                             d_X+j, D, d_q, M, &beta, d_W, D);

          
          // Note: If you get an error messages concerning SGEMM 
          //       (instead of cublasSgemm), know that cublasSgemm calls
          //       SGEMM as follows:
          // SGEMM(TRANSA, TRANSB, M, N, K, ALPHA, A, LDA, B, LDB, BETA, C, LDC)
        
        }
        end = clock();
        float time1 = ((float)(end-start))/CLOCKS_PER_SEC;
        printf("Duree: %f seconds\n",time1);


        J /= -N_train;

        // Compute the classification score
        if (i%(nb_iter/periods)==0) {
          printf("iter: %d, logloss: %f\n",i,J);
        
        }
    }
    
    /* Read the result back */
    status = cublasGetVector(nW, sizeof(h_W[0]), d_W, 1, h_W, 1);

    int evaluate_accuracy = 1;
    int true_class = 0;
    int nb_tested = 0;
    if (evaluate_accuracy) {
        ////////////////////////////////
        // compute Z = W^T X'
        // --> each column of Z corresponds to one input
        ////////////////////////////////
        alpha = 1.0;
        beta = 0.0;
        status = cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_N, 
                             M, N, D, &alpha, 
                             d_W, D, d_X, D, &beta, d_Z, M);

        // On the host:
        // Copy Z back to host
        status = cublasGetVector(nZ, sizeof(h_Z[0]), d_Z, 1, h_Z, 1);

        ////////////////////////////////
        // For each column z of Z, 
        // compute p
        // then find argmax_k p_k
        ////////////////////////////////
        float logloss = 0.0;
        // Note: we start after the training samples, 
        //       which go from 0 to N_train-1
        for (j=N_train; j < N; ++j) {
          // To check accuracy for all samples: 
          //  for (j=0; j < N; ++j) {
          ////////////////////////////////
          // For the k-th element of z,
          // compute p_k = exp(z_k)/sum_t exp(z_t)
          ////////////////////////////////
          // compute sumz = sum_t exp(z_t)
          float sumz = 0.0;
          for (k=0; k < M; ++k) {
             sumz += exp(h_Z[IDX2C(k,j,M)]);
          }
          float p_max = 0;
          size_t k_max = 0;
          for (k=0; k < M; ++k) {
             float p_k = exp(h_Z[IDX2C(k,j,M)])/sumz;
             if (p_k>p_max) {
                 p_max = p_k;
                 k_max = k;
             }
             if (nb_tested < print_rows) {
                 printf("p_%d=%f,y_%d=%f ",k,p_k,k,Y[IDX2C(j,k,N)]);
             }
          }
          if (nb_tested < print_rows) {
              printf("max: p_%d=%f,y_%d=%f ",k_max,p_max,k,Y[IDX2C(j,k_max,N)]);
              printf("\n");
          }
          // compare prediction to label
          if (Y[IDX2C(j,k_max,N)]>=0.5f) {
              ++true_class;
          }
          ++nb_tested;
        }
    }
    
    printf("Correct results: %d out of %d\n",true_class,nb_tested);
    printf("Accuracy: %f\n",(float)true_class/(float)nb_tested);
        
    /* Memory clean up */
    free(h_Z);
    free(h_W);
    free(h_q);

    cudaFree(d_q);
    cudaFree(d_X);
    cudaFree(d_W);
    cudaFree(d_Z);

    /* Shutdown */
    status = cublasDestroy(handle);
}

headers: "longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"!
Read 3000 rows.
Data (first 10 rows):
-114.31	34.19	15	5612	1283	1015	472	1.4936	66900	
-114.47	34.4	19	7650	1901	1129	463	1.82	80100	
-114.56	33.69	17	720	174	333	117	1.6509	85700	
-114.57	33.64	14	1501	337	515	226	3.1917	73400	
-114.57	33.57	20	1454	326	624	262	1.925	65500	
-114.58	33.63	29	1387	236	671	239	3.3438	74000	
-114.58	33.61	25	2907	680	1841	633	2.6768	82400	
-114.59	34.83	41	812	168	375	158	1.7083	48500	
-114.59	33.61	34	4789	1175	3134	1056	2.1782	58400	
-114.6	34.83	46	1497	309	787	271	2.1908	48100	
Allocated memory: 3000 rows, 9 columns.
Inputs (first 10):
1	1	1	1	1	1	1	1	1	1	
-114.31	-114.47	-114.56	-114.57	-114.57	-114.58	-114.58	-114.59	-114.59	-114.6	
34.19	34.4	33.69	33.64	33.57	33.63	33.61	34.83	33.61	34.83	
15	19	17	14	20	29	25	41	34	46	
5612	7650	720	1501	1454	1387	2907	812	4789	1497	
1283	1901	174	337	326	236	680	168

For this version, the hyper parameter we can change is the learning rate.
With a learning rate of 0.000001, I have an accuracy of 77.9%
Un learning rate plus faible ou plus fort a une accuracy moidre car la convergence est moins bonne.
Au dela de 10 iterations, l'accuracy diminue car on overfit. En dessous de 10 iterations, on underfit. 

### Kernel method

In [78]:
%%cu 
/** CUBLAS example taken from http://courses.cms.caltech.edu/cs179/ */

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#include <iostream>
#include <iomanip>
#include <math.h>
#include <fstream>

/* Includes, cuda */
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <helper_cuda.h>

using namespace std;


/* transform matrix index to vector offset
   Since CUDA uses column major, 
   ld = number of rows 
   Example of use: a[IDX2C(0, 1, 50)] */
#define IDX2C(i,j,ld) (((j)*(ld))+(i))

/* Constants for housing data set */
#define data_columns  (9)
#define above_threshold (265000.0)

/////////////////////////////////////////////////////////
// Number of rows in arrays to print for debugging
/////////////////////////////////////////////////////////
#define print_rows (10)

/////////////////////////////////////////////////////////
// Auxiliary function
/////////////////////////////////////////////////////////
// generate random numbers in interval [min,max]
float float_rand( float min, float max )
{
    float scale = rand() / (float) RAND_MAX; /* [0, 1.0] */
    return min + scale * ( max - min );      /* [min, max] */
}


/////////////////////////////////////////////////////////
// Functions for reading the dataset from a file
/////////////////////////////////////////////////////////

/* Read a csv file with a given number of rows and columns */
void read_csv(const char* filename, float* data_array,size_t nbrow,size_t nbcol) {
  string row_as_string;
  string value;
  double ioTemp;
  ifstream infile;
  infile.open(filename, ifstream::in);
  size_t row_count = 0;
	if (infile.is_open())
  {
      // read the headers (and discard)
			getline(infile, row_as_string, '\n');
      cout << "headers: " << row_as_string << "!" << std::endl;
      for(int i = 0; i < nbrow; i++){
  			getline(infile, row_as_string, '\n');
        // cout << "read line " << row_as_string << "!" << std::endl;
				istringstream line_stream(row_as_string);
			  for(int j = 0; j < nbcol; j++){
          getline(line_stream, value, ',');
					ioTemp = strtod(value.c_str(), NULL); 
          // cout << "("<<i<<","<<j<<") = "<< ioTemp << std::endl;

					data_array[IDX2C(i,j,nbrow)] = ioTemp;

				}
        ++row_count;
			}
		infile.close();
    cout << "Read " << row_count << " rows." << std::endl;
	}
	else cout << "Cannot open file." << endl;
}

/* Allocate memory and fill with given number of rows from housing file */
void read_housing_csv(float** data_array, size_t nbrows) {
    *data_array = (float *)malloc(nbrows * data_columns * sizeof(float));
    
    read_csv("sample_data/california_housing_train.csv",*data_array,nbrows,data_columns);

    cout << "Data (first "<<print_rows<<" rows):" << std::endl;
    // Show some entries for double checking
    for(int i = 0; i < nbrows && i < print_rows; i++){
			for(int j = 0; j < data_columns; j++){
				cout << (*data_array)[IDX2C(i,j,nbrows)] << "\t";
			}
      cout << "\n";
		}
}

/////////////////////////////////////////////////////////
// Functions for preprocessing the data set
/////////////////////////////////////////////////////////

/* Split data into inputs and labels. Allocated memory for inputs and labels.
   Since cuBLAS is column major, each input is in a column.
   We also add 1.0 as first element to each input vector.
*/
void get_inputs_and_labels(float* data_array, float** input_array, float** label_array, size_t nbrows, size_t nbcols, size_t* nb_inputs, size_t* nb_labels ) {
    // The inputs are the first nbrows-1 columns.
    // The labels are the last column (index nbrows-1), booleanized
    // by the condition >= above_threshold
    size_t nbcols_input = nbcols-1+1; // remove one column, add one for 1.0
    size_t nbcols_labels = 2;
    *nb_inputs = nbcols_input;
    *nb_labels = nbcols_labels;
    *input_array = (float *)malloc(nbrows * nbcols_input * sizeof(float));    
    *label_array = (float *)malloc(nbrows * nbcols_labels * sizeof(float));    
    //cout << &input_array << " and "<< &label_array << " data " << data_array << std::endl;
    cout << "Allocated memory: " << nbrows << " rows, "<< nbcols_input << " columns." << std::endl;

    // Copy the data to X
    for(int i = 0; i < nbrows; i++){
      // Set the first element of each x to 1  
      (*input_array)[IDX2C(0,i,nbcols_input)] = 1.0;
      // Copy the rest of x
			for(int j = 1; j < nbcols_input; j++){
				(*input_array)[IDX2C(j,i,nbcols_input)] = data_array[IDX2C(i,j-1,nbrows)];
			}
      float median_house_value = data_array[IDX2C(i,nbcols-1,nbrows)];
      (*label_array)[IDX2C(i,0,nbrows)] = 0.0;
      (*label_array)[IDX2C(i,1,nbrows)] = 0.0;
      if (median_house_value >= above_threshold) {
        (*label_array)[IDX2C(i,0,nbrows)] = 1.0;
      } else {
        (*label_array)[IDX2C(i,1,nbrows)] = 1.0;        
      }
		}    
    
    // Show some entries for double checking
    cout << "Inputs (first "<<print_rows<<"):" << std::endl;
	  for(int j = 0; j < nbcols_input; j++){
      for(int i = 0; i < nbrows && i < print_rows; i++){
				cout << (*input_array)[IDX2C(j,i,nbcols_input)] << "\t";
			}
      cout << "\n";
		}
    cout << "Labels (first "<<print_rows<<"):" << std::endl;
    for(int i = 0; i < nbrows && i < print_rows; i++){
			for(int j = 0; j < nbcols_labels; j++){
				cout << (*label_array)[IDX2C(i,j,nbrows)] << "\t";
			}
      cout << "\n";
		}
}

__global__ 
void kernel(float *d_q, float *d_z, float *d_y , int j, int M, int N) {
    float sumz = 0.0;       
    for (int k=0; k < M; ++k) {
        sumz += exp(d_z[IDX2C(k,j,M)]);}
 
    for (int k=0; k < M; ++k) {
        float p_k = exp(d_z[IDX2C(k,j,M)])/sumz;
        float q_k = p_k - d_y[IDX2C(j,k,N)];
        d_q[k] = q_k;
        }
      }



/////////////////////////////////////////////////////////
// Main program
/////////////////////////////////////////////////////////
int main(int argc, char **argv)
{
    
    /////////////////////////////////////////////////////////
    // Parameters for the data set
    /////////////////////////////////////////////////////////
    size_t N_train = 2000; // points for training (Google: 12000)
    size_t N_test = 1000; // points for validation (Google: 5000)
    size_t N = N_train+N_test;
    /////////////////////////////////////////////////////////
    // Reading the data set
    /////////////////////////////////////////////////////////
    float *alldata = 0;
    read_housing_csv(&alldata,N);
    float* X = 0; 
    float* Y = 0;
    size_t D; // number of inputs
    size_t M; // number of labels
    get_inputs_and_labels(alldata,&X,&Y,N,data_columns,&D,&M);
    /////////////////////////////////////////////////////////
    // Inputs and labels are now available in X and Y.
    // Each input is a column in X; each feature is a row;
    //    by convention, the first row has the value 1.0;
    //    X is of dimension D x N
    // each label is a row in Y; Y is of dimension N x M
    /////////////////////////////////////////////////////////

    
    /////////////////////////////////////////////////////////
    // Parameters for Stochastic Gradient Descent
    /////////////////////////////////////////////////////////
    int nb_iter = 10;
    int periods = nb_iter; // reporting periods
    float step_size = 0.000001;
    
    
    /////////////////////////////////////////////////////////
    // Memory Allocation and Initialization
    /////////////////////////////////////////////////////////
    cublasStatus_t status;
    float *h_W;
    float *h_Z;
    float *h_q;
    float *h_y;
    float *d_X = 0;
    float *d_W = 0;
    float *d_Z = 0;
    float *d_q = 0;
    float *d_y = 0;
    float alpha = 1.0f;
    float beta = 0.0f;
    size_t nX = D * N; // nb of elements in X
    size_t nW = D * M; // nb of elements in W
    size_t nZ = M * N; // nb of elements in Z=W^TX
    int i,j,k;
    float J;
    cublasHandle_t handle;

    status = cublasCreate(&handle);

    /* Allocate host memory for the matrices */
    h_Z = (float *)malloc(nZ * sizeof(h_Z[0]));
    h_q = (float *)malloc(M * sizeof(h_q[0]));   
    h_y = (float *)malloc(M * sizeof(h_y[0]));


    /////////////////////////////////////////////////////////
    // Initializing Weight Matrix 
    // its dimension is D x M
    /////////////////////////////////////////////////////////
    /* Find scale for each input */
    float scale[D];
    for (int i = 0; i < D; ++i) {
        scale[i] = 0.0;
        for (int j = 0; j < N_train; ++j) {
            scale[i] = max(scale[i],abs(X[IDX2C(i,j,D)]));
        }
    }

    /* Initialize Weight Matrix,
       scaling each row according to the magnitude of its
       corresponding feature.
       Its dimension is D * M */
    h_W = (float *)malloc(nW * sizeof(h_W[0]));
    for (int i = 0; i < D; ++i) {
        for (int j = 0; j < M; ++j) {
            h_W[IDX2C(i,j,D)] = 1.0/sqrt(D) * float_rand( -1, 1 ) / scale[i];
        }
    }

    /* Allocate device memory for the matrices */
    cudaMalloc((void **)&d_Z, nZ * sizeof(d_Z[0]));
    cudaMalloc((void **)&d_X, nX * sizeof(d_X[0]));
    cudaMalloc((void **)&d_W, nW * sizeof(d_W[0]));
    cudaMalloc((void **)&d_q, M * sizeof(d_q[0]));\
    cudaMalloc((void **)&d_y, M * sizeof(d_q[0]));


    /* Initialize the device matrices with the host matrices */
    status = cublasSetVector(nX, sizeof(X[0]), X, 1, d_X, 1);
    status = cublasSetVector(nW, sizeof(h_W[0]), h_W, 1, d_W, 1);

    /////////////////////////////////////////////////////////
    // Batch Gradient Descent
    /////////////////////////////////////////////////////////
    for (i = 0; i < nb_iter; ++i ) {
        ////////////////////////////////
        // compute Z = W^T X
        // --> each column z of Z corresponds to one column x of X
        ////////////////////////////////
        // Reminder: 
        // W^T has dimensions M by D
        // X has dimensions D by N
        // Z has dimensions M by N
        
        /* ... to be completed ... */

        status = cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_N, 
                             M, N, D, &alpha, 
                             d_W, D, d_X, D, &beta, d_Z, M);

        ////////////////////////////////
        // Compute softmax activation p(z)
        // Version 1: on the host
        ////////////////////////////////
        // Copy Z back to host
        status = cublasGetVector(nZ, sizeof(h_Z[0]), d_Z, 1, h_Z, 1);
          
        ////////////////////////////////
        // For each column z of Z, compute p;
        // then update W
        ////////////////////////////////
         #include <time.h>
        clock_t start,end;
        start = clock();

        J = 0.0;
        for (j=0; j < N_train; ++j) {
          ////////////////////////////////
          // For the k-th element of z,
          // compute p_k = exp(z_k)/sum_t exp(z_t)
          //         q_k = p_k - y_k(j)
          ////////////////////////////////
        
          /* ... compute q_k and store in array h_q[k] ...
            ... to be completed ... */

          for (int k=0; k < M; ++k) {
              h_y[k] = Y[IDX2C(j,k,N)];           
          }


          cudaMemcpy(d_y, h_y, sizeof(h_y[0])*M, cudaMemcpyHostToDevice);
          kernel<<<1,M>>>(d_q,d_Z,d_y,j,M,N);
  

          // Copy q onto device
          status = cublasSetVector(M, sizeof(h_q[0]), h_q, 1, d_q, 1);
            
          ////////////////////////////////
          // Compute the LogLoss score
          // J = - sum_j sum_k y_k * log(p_k) / N_train
          ////////////////////////////////
            
          ////////////////////////////////
          // compute  d = x(p-y)^T;
          // update   W = W - a/N_train * d
          ////////////////////////////////
  
          /* ... to be completed ... */

          alpha = - step_size / N_train ;
          beta = 1;

          status = cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_T, 
                             D, M, 1, &alpha, 
                             d_X+j, D, d_q, M, &beta, d_W, D);

          
          // Note: If you get an error messages concerning SGEMM 
          //       (instead of cublasSgemm), know that cublasSgemm calls
          //       SGEMM as follows:
          // SGEMM(TRANSA, TRANSB, M, N, K, ALPHA, A, LDA, B, LDB, BETA, C, LDC)
        
        }
        end = clock();
        float time1 = ((float)(end-start))/CLOCKS_PER_SEC;
        printf("Duree: %f seconds\n",time1);
//        J /= -N_train;

        // Compute the classification score
        if (i%(nb_iter/periods)==0) {
          printf("iter: %d, logloss: %f\n",i,J);
        
        }
    }
    
    /* Read the result back */
    status = cublasGetVector(nW, sizeof(h_W[0]), d_W, 1, h_W, 1);

    int evaluate_accuracy = 1;
    int true_class = 0;
    int nb_tested = 0;
    if (evaluate_accuracy) {
        ////////////////////////////////
        // compute Z = W^T X'
        // --> each column of Z corresponds to one input
        ////////////////////////////////
        alpha = 1.0;
        beta = 0.0;
        status = cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_N, 
                             M, N, D, &alpha, 
                             d_W, D, d_X, D, &beta, d_Z, M);

        // On the host:
        // Copy Z back to host
        status = cublasGetVector(nZ, sizeof(h_Z[0]), d_Z, 1, h_Z, 1);

        ////////////////////////////////
        // For each column z of Z, 
        // compute p
        // then find argmax_k p_k
        ////////////////////////////////
        float logloss = 0.0;
        // Note: we start after the training samples, 
        //       which go from 0 to N_train-1
        for (j=N_train; j < N; ++j) {
          // To check accuracy for all samples: 
          //  for (j=0; j < N; ++j) {
          ////////////////////////////////
          // For the k-th element of z,
          // compute p_k = exp(z_k)/sum_t exp(z_t)
          ////////////////////////////////
          // compute sumz = sum_t exp(z_t)
          float sumz = 0.0;
          for (k=0; k < M; ++k) {
             sumz += exp(h_Z[IDX2C(k,j,M)]);
          }
          float p_max = 0;
          size_t k_max = 0;
          for (k=0; k < M; ++k) {
             float p_k = exp(h_Z[IDX2C(k,j,M)])/sumz;
             if (p_k>p_max) {
                 p_max = p_k;
                 k_max = k;
             }
             if (nb_tested < print_rows) {
                 printf("p_%d=%f,y_%d=%f ",k,p_k,k,Y[IDX2C(j,k,N)]);
             }
          }
          if (nb_tested < print_rows) {
              printf("max: p_%d=%f,y_%d=%f ",k_max,p_max,k,Y[IDX2C(j,k_max,N)]);
              printf("\n");
          }
          // compare prediction to label
          if (Y[IDX2C(j,k_max,N)]>=0.5f) {
              ++true_class;
          }
          ++nb_tested;
        }
    }
    
    printf("Correct results: %d out of %d\n",true_class,nb_tested);
    printf("Accuracy: %f\n",(float)true_class/(float)nb_tested);
        
    /* Memory clean up */
    free(h_Z);
    free(h_W);
    free(h_q);

    cudaFree(d_q);
    cudaFree(d_X);
    cudaFree(d_W);
    cudaFree(d_Z);

    /* Shutdown */
    status = cublasDestroy(handle);
}

headers: "longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"!
Read 3000 rows.
Data (first 10 rows):
-114.31	34.19	15	5612	1283	1015	472	1.4936	66900	
-114.47	34.4	19	7650	1901	1129	463	1.82	80100	
-114.56	33.69	17	720	174	333	117	1.6509	85700	
-114.57	33.64	14	1501	337	515	226	3.1917	73400	
-114.57	33.57	20	1454	326	624	262	1.925	65500	
-114.58	33.63	29	1387	236	671	239	3.3438	74000	
-114.58	33.61	25	2907	680	1841	633	2.6768	82400	
-114.59	34.83	41	812	168	375	158	1.7083	48500	
-114.59	33.61	34	4789	1175	3134	1056	2.1782	58400	
-114.6	34.83	46	1497	309	787	271	2.1908	48100	
Allocated memory: 3000 rows, 9 columns.
Inputs (first 10):
1	1	1	1	1	1	1	1	1	1	
-114.31	-114.47	-114.56	-114.57	-114.57	-114.58	-114.58	-114.59	-114.59	-114.6	
34.19	34.4	33.69	33.64	33.57	33.63	33.61	34.83	33.61	34.83	
15	19	17	14	20	29	25	41	34	46	
5612	7650	720	1501	1454	1387	2907	812	4789	1497	
1283	1901	174	337	326	236	680	168

L'utilisation du kernel pour faire ce calcul ne permet pas d'ameliorer le temps de calcul mais en combinant cela avec des mini natch, om pourrait voir une amelioration.